In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import json
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
rumah_sakit = pd.read_csv("C:/Users/LENOVO/Downloads/Rumah_Sakit.csv")
pemadam = pd.read_csv("C:/Users/LENOVO/Downloads/Pemadam.csv")
polisi = pd.read_csv("C:/Users/LENOVO/Downloads/Polisi.csv")

In [ ]:
emergency = pd.concat([rumah_sakit, pemadam, polisi], ignore_index = True)

In [ ]:
emergency

In [ ]:
types_dummies = pd.get_dummies(emergency['types'].str.get_dummies(','), prefix='types', prefix_sep='_')
types_dummies

In [ ]:
df_all = pd.concat([emergency, types_dummies], axis=1)
df_all = df_all.drop(columns = ["point_of_interest", "point_of_interest", "latlong"])
df_all = pd.merge(restaurant_review, df_all, on = "place_id")
df_all.rename(columns={'rating': 'Ave_Rating'}, inplace=True)
df_all['reviewers_Id'] = pd.factorize(df_all['reviewer_name'])[0] + 1
df_all

In [ ]:
emergency['types'] = emergency['types'].str.lower()
emergency.loc[emergency['types'].str.contains('hospital'), 'types'] = 'hospital'
emergency.loc[emergency['types'].str.contains('police'), 'types'] = 'police'
emergency.loc[emergency['types'].str.contains('fire_station'), 'types'] = 'fire_station'

In [ ]:
emergency

In [ ]:
emergency = emergency[emergency['types'].isin(['police', 'fire_station', 'hospital'])]

In [ ]:
emergency

In [ ]:
emergency = emergency.drop(columns = ["Unnamed: 0", "latlong"])

In [ ]:
emergency

In [ ]:
place_id = emergency.pop("place_id")
emergency.insert(0, 'place_id', place_id)
emergency

In [ ]:
emergency.rename(columns={'name': 'place_name'}, inplace=True)
emergency.rename(columns={'formatted_address': 'place_address'}, inplace=True)
emergency.rename(columns={'types': 'place_category'}, inplace=True)
emergency.rename(columns={'user_ratings_total': 'total_review'}, inplace=True)
emergency

In [ ]:
emergency.rename(columns={'rating': 'ave_rating'}, inplace=True)
emergency

In [ ]:
emergency = emergency.drop_duplicates()
emergency = emergency.reset_index()
emergency

In [ ]:
emergency = emergency.drop(columns = "index")

In [ ]:
emergency

In [ ]:
emergency.to_csv('D:/Project_Jejaka/emergency/emergency.csv', index=False)

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import json
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
app = Flask(__name__)

In [ ]:
class Emergency:
    def __init__(self):
        self.emergency = None
        self.tfidf_matrix_emergency = None
        self.similarity_tensor = None
        self.load_data("D:/Project_Jejaka/emergency/emergency.csv")
        self.tfidf_matrix_emergency, self.tfidf = self.TF_IDF()
        self.calculate_similarity(self.tfidf_matrix_emergency)
        
    def load_data(self, filename):
        self.emergency = pd.read_csv(filename)
        
    def preprocess_text(self):
        combined_text = self.emergency['place_name'] + ' ' + self.emergency['place_category'] + ' ' + self.emergency['place_address']
        return combined_text
    
    def TF_IDF(self):
        tfidf = TfidfVectorizer()
        combined_text = self.preprocess_text()
        tfidf_matrix = tfidf.fit_transform(combined_text)
        return tfidf_matrix, tfidf
    
    def calculate_similarity(self, tfidf_matrix):
        similarity_matrix = cosine_similarity(tfidf_matrix)
        self.similarity_tensor = tf.convert_to_tensor(similarity_matrix)
    
    def recommend(self, input_text, k):
        tfidf_matrix, tfidf = self.TF_IDF()
        
        tfidf_matrix_input = tfidf.transform([input_text])
        similarity_scores = cosine_similarity(tfidf_matrix_input, tfidf_matrix)
        similarity_scores = similarity_scores.flatten()
        
        top_k_values, top_k_indices = tf.nn.top_k(similarity_scores, k=k)
        top_k_places = self.emergency.loc[top_k_indices]
        
        return top_k_places

emer = Emergency()

In [ ]:
emer = Emergency()

In [ ]:
@app.route('/recommend-emergency', methods=['POST'])
def recommend_emergency():
    data = request.json
    input_text = data['input_text']
    k = data['k']

    emergency_places = emer.recommend(input_text, k)
    recommended_places_json = emergency_places.to_dict(orient='records')

    return jsonify({'places': recommended_places_json})

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf

def load_data(filename):
    return pd.read_csv(filename)

def preprocess_text_hotel(data):
    combined_text = data['place_name'] + ' ' + data['desc'] + ' ' + data['place_address']
    return combined_text
    variables = ['art_gallery', 'bakery', 'bar', 'cafe', 'clothing_store', 'electronics_store',
                 'food', 'gym', 'health', 'home_goods_store', 'lodging', 'night_club',
                 'park', 'parking', 'real_estate_agency', 'restaurant', 'shopping_mall',
                 'spa', 'store', 'travel_agency']
    
    for variable in variables:
        combined_text += data[data[variable] == 1][variable].apply(lambda x: ' ' + variable)
    
    return combined_text

def preprocess_text_restaurant(data):
    combined_text = data['place_name'] + ' ' + data['desc'] + ' ' + data['place_address']
    return combined_text
    variables = ['art_gallery', 'bakery', 'bar', 'cafe', 'food', 'liquor_store', 'lodging',
                 'meal_delivery', 'meal_takeaway', 'night_club', 'restaurant', 'school', 'store']
    
    for variable in variables:
        combined_text += data[data[variable] == 1][variable].apply(lambda x: ' ' + variable)
    
    return combined_text

def preprocess_text_tourism(data):
    combined_text = data['place_name'] + ' ' + data['desc'] + ' ' + data['place_address']
    return combined_text
    variables = ['amusement_park', 'aquarium', 'art_gallery', 'cafe', 'church', 'food',
                 'hindu_temple', 'library', 'local_government_office', 'lodging', 'mosque',
                 'museum', 'natural_feature', 'park', 'place_of_worship', 'restaurant',
                 'rv_park', 'school', 'shopping_mall', 'store', 'travel_agency', 'zoo']
    
    for variable in variables:
        combined_text += data[data[variable] == 1][variable].apply(lambda x: ' ' + variable)
    
    return combined_text

def calculate_tfidf_matrix(combined_text):
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(combined_text)
    return tfidf_matrix, tfidf

tourism_place = load_data("D:/Project_Jejaka/tourism/item_data.csv")
tourism_combined_text = preprocess_text_tourism(tourism_place)
tfidf_matrix_tourism, tfidf_tourism = calculate_tfidf_matrix(tourism_combined_text)

restaurant_place = load_data("D:/Project_Jejaka/restaurant/item_data.csv")
restaurant_combined_text = preprocess_text_restaurant(restaurant_place)
tfidf_matrix_restaurant, tfidf_restaurant = calculate_tfidf_matrix(restaurant_combined_text)

hotel_place = load_data("D:/Project_Jejaka/hotel/item_data.csv")
hotel_combined_text = preprocess_text_hotel(hotel_place)
tfidf_matrix_hotel, tfidf_hotel = calculate_tfidf_matrix(hotel_combined_text)

In [ ]:
tourism_combined_text

In [ ]:
tfidf_matrix_tourism, tfidf_tourism = calculate_tfidf_matrix(tourism_combined_text)
tourism_similarity_tensor = calculate_similarity(tfidf_matrix_tourism)

# Load restaurant data and perform necessary operations
restaurant_place = load_data("D:/Project_Jejaka/restaurant/item_data.csv")
restaurant_combined_text = preprocess_text_restaurant(restaurant_place)
tfidf_matrix_restaurant, tfidf_restaurant = calculate_tfidf_matrix(restaurant_combined_text)
restaurant_similarity_tensor = calculate_similarity(tfidf_matrix_restaurant)

# Load hotel data and perform necessary operations
hotel_place = load_data("D:/Project_Jejaka/hotel/item_data.csv")
hotel_combined_text = preprocess_text_hotel(hotel_place)
tfidf_matrix_hotel, tfidf_hotel = calculate_tfidf_matrix(hotel_combined_text)

In [ ]:
tfidf_matrix_hotel

In [ ]:
import joblib

In [ ]:
joblib.dump(tfidf_restaurant, 'D:/Project_Jejaka/search-bar/tfidf_vectorizer_restaurant.joblib')
joblib.dump(tfidf_hotel, 'D:/Project_Jejaka/search-bar/tfidf_vectorizer_hotel.joblib')
joblib.dump(tfidf_tourism, 'D:/Project_Jejaka/search-bar/tfidf_vectorizer_tourism.joblib')

In [ ]:
joblib.dump(tfidf_matrix_restaurant, 'D:/Project_Jejaka/search-bar/tfidf_matrix_restaurant.joblib')
joblib.dump(tfidf_matrix_hotel, 'D:/Project_Jejaka/search-bar/tfidf_matrix_hotel.joblib')
joblib.dump(tfidf_matrix_tourism, 'D:/Project_Jejaka/search-bar/tfidf_matrix_tourism.joblib')

In [ ]:
a = joblib.load('D:/Project_Jejaka/search-bar/tfidf_matrix_hotel.joblib')
b = joblib.load('D:/Project_Jejaka/search-bar/tfidf_matrix_restaurant.joblib')
c = joblib.load('D:/Project_Jejaka/search-bar/tfidf_matrix_tourism.joblib')

In [ ]:
combined_text = tourism_combined_text + restaurant_combined_text + hotel_combined_text

In [ ]:
combined_text

In [ ]:
hotel_combined_text

In [ ]:
d = tourism_place['place_name'] + ' ' + tourism_place['desc'] + ' ' + tourism_place['place_address']

In [ ]:
d

In [ ]:
import pandas as pd

In [ ]:
item_data = pd.read_csv("D:/Project_Jejaka/hotel/item_data.csv")

In [ ]:
item_data

In [ ]:
# Fungsi untuk memproses alamat
def process_address(address):
    city_mapping = {
        'yogyakarta': 'Yogyakarta',
        'bandung': 'Bandung',
        'malang': 'Malang',
        'padang': 'Padang',
        'medan': 'Medan',
        'bali': 'Bali',
        'jakarta': 'Jakarta',
        'lombok': 'Lombok',
        'surakarta': 'Surakarta',
        'batam': 'Batam',
    }
    
    address_lower = address.lower()
    
    for city, city_name in city_mapping.items():
        if city in address_lower:
            return city_name
    
    return ''

# Menambahkan kolom baru "city" berdasarkan alamat
item_data.insert(item_data.columns.get_loc('place_address') + 1, 'city/regency', item_data['place_address'].apply(process_address))

# Menampilkan DataFrame hasil
item_data


In [ ]:
item_data["city/regency"].value_counts()

In [ ]:
item_data.to_csv('item_data_hotel.csv', index=False)

In [ ]:
item_data = pd.read_csv("C:/Users/LENOVO/item_data_restaurant.csv", sep = ";")

In [ ]:
item_data

In [ ]:
item_data.to_csv('item_data_restaurant.csv', index=False)

In [2]:
import os
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.cloud import bigquery

app = Flask(__name__)

# Mengatur kredensial BigQuery
credentials_path = "direct-hope-387806-8d2b15781824.json"  # Ganti dengan path file JSON kunci yang Anda unduh

# Membuat objek koneksi BigQuery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
client = bigquery.Client()

# Load item data
item_data_tourism = None
item_data_restaurant = None
item_data_hotel = None

def load_data():
    global item_data_tourism, item_data_restaurant, item_data_hotel
    
    # Load item data from BigQuery
    item_data_tourism = load_data_from_bigquery('item_data_tourism', ['place_id', 'place_name', 'desc', 'place_address', 'total_review', 'ave_rating'])
    item_data_restaurant = load_data_from_bigquery('item_data_restaurant', ['place_id', 'place_name', 'desc', 'place_address', 'total_review', 'ave_rating'])
    item_data_hotel = load_data_from_bigquery('item_data_hotel', ['place_id', 'place_name', 'desc', 'place_address', 'total_review', 'ave_rating'])
    
    item_data_tourism['city'] = item_data_tourism['place_address'].apply(lambda x: x.split()[-1])
    item_data_restaurant['city'] = item_data_restaurant['place_address'].apply(lambda x: x.split()[-1])
    item_data_hotel['city'] = item_data_hotel['place_address'].apply(lambda x: x.split()[-1])

def load_data_from_bigquery(table_name, selected_columns=None):
    table_ref = client.dataset('Jejaka').table(table_name)
    table = client.get_table(table_ref)
    
    if selected_columns is None:
        data = client.list_rows(table).to_dataframe()
    else:
        schema = table.schema
        selected_fields = [field for field in schema if field.name in selected_columns]
        data = client.list_rows(table, selected_fields=selected_fields).to_dataframe()
    
    return data

@app.route('/search', methods=['POST'])
def search():
    # Get input text and filters from the request
    data = request.get_json()
    input_text = data.get('input_text', '')
    filters = data.get('filters', [])  # Use a list of filters

    # Combine the item data based on the selected filters
    if 'tourism' in filters:
        item_data = item_data_tourism
    if 'restaurant' in filters:
        item_data = item_data_restaurant
    if 'hotel' in filters:
        item_data = item_data_hotel

    # Create a TF-IDF vectorizer and fit it on the item data
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(item_data['place_name'] + ' ' + item_data['desc'] + ' ' + item_data['place_address'])

    # Transform input text using TF-IDF vectorizer
    tfidf_matrix_input = tfidf_vectorizer.transform([input_text])

    # Perform cosine similarity and get top-k results
    similarity_scores = cosine_similarity(tfidf_matrix_input, tfidf_matrix)
    similarity_scores = similarity_scores.flatten()
    top_k_indices = np.argsort(similarity_scores)[::-1][:20]
    top_k_places = item_data.iloc[top_k_indices]

    # Convert the results to a JSON response
    results = []
    for index, row in top_k_places.iterrows():
        result = {
            'place_id': row['place_id'],
            'place_name': row['place_name'],
            'place_address': row['place_address'],
            'total_review': row['total_review'],
            'ave_rating': row['ave_rating']
        }
        results.append(result)

    return jsonify({'results': results})

if __name__ == '__main__':
    load_data()
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
